In [1]:
!pip install beautifulsoup4

In [1]:
!pip install selenium

In [49]:
#https://builtin.com/articles/selenium-web-scraping
#https://brightdata.com/blog/how-tos/using-selenium-for-web-scraping#:~:text=Before%20you%20start%20web%20scraping,Chrome%2C%20Firefox%2C%20and%20Edge.

#Import selenium and other required packages for the scrape
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [56]:
#Set up the chrome driver options
options = Options()
options.add_argument("--headless")

#Set up the driver
service = Service("C:/Users/Jane Barraclough/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [57]:
#Get the url
url = "https://howstat.com/Cricket/Statistics/Matches/MatchListCountry.asp?A=IND&B=XXX&C=XXX"
driver.get(url)

#Get the full HTML and close the browser
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

In [58]:
#I know I need table 5 for my analysis from the previous scrape attempt
tables = soup.find_all('table')
match_table = tables[5]
rows = match_table.find_all('tr')

In [59]:
#Need to confirm the number of rows matches the matches played
print(len(rows))

590


In [60]:
# Get the column names
header_row = rows[0]
headers = [cell.text.strip() for cell in header_row.find_all(['th', 'td'])]
print(headers)

['', 'Date', 'Series', 'Ground', 'Result']


In [62]:
# Extract all match data rows
data_rows = []

for row in rows[1:]:
    cells = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
    if cells:
        data_rows.append(cells)

print(data_rows[0:4])


[['1', '25/06/1932', '1932 England v. India', "Lord's", 'England won by 158 runs'], ['2', '15/12/1933', '1933-1934 India v. England', 'Gymkhana Ground', 'England won by 9 wickets'], ['3', '05/01/1934', '1933-1934 India v. England', 'Eden Gardens', 'Match Drawn'], ['4', '10/02/1934', '1933-1934 India v. England', 'MA Chidambaram Stadium', 'England won by 202 runs']]


In [63]:
df = pd.DataFrame(data_rows, columns=headers)
df.head()

,,Date,Series,Ground,Result
0,1,25/06/1932,1932 England v. India,Lord's,England won by 158 runs
1,2,15/12/1933,1933-1934 India v. England,Gymkhana Ground,England won by 9 wickets
2,3,05/01/1934,1933-1934 India v. England,Eden Gardens,Match Drawn
3,4,10/02/1934,1933-1934 India v. England,MA Chidambaram Stadium,England won by 202 runs
4,5,27/06/1936,1936 England v. India,Lord's,England won by 9 wickets


In [65]:
df.columns = ['Test Number', 'Date', 'Series', 'Ground', 'Result']
df.head()

,Test Number,Date,Series,Ground,Result
0,1,25/06/1932,1932 England v. India,Lord's,England won by 158 runs
1,2,15/12/1933,1933-1934 India v. England,Gymkhana Ground,England won by 9 wickets
2,3,05/01/1934,1933-1934 India v. England,Eden Gardens,Match Drawn
3,4,10/02/1934,1933-1934 India v. England,MA Chidambaram Stadium,England won by 202 runs
4,5,27/06/1936,1936 England v. India,Lord's,England won by 9 wickets


In [68]:
#Download to a CSV file
df.to_csv('../data/india_match_results.csv', index=False)